# Background Preparation


**目录：**
1. bisect Module
2. Sum Tree
3. Duel Structure
---

## 1 bisect Module
### 1) 二分查找

bisect是python自带的二分查找的包，默认应用于increasing array。

increasing array:
    
    如果一个array,`arr`满足 `arr[i1]` $\le$ `arr[i2]` if `i1` $\le$ `i2`
    
二分查找:

    一种在有序数列中查找一个元素的高效的方法，每次将数组以中间为分割点，分为两部分，然后进入相应的部分进行查找。
    
    以increasing array为例，要在`arr`中寻找`num`，先比较`num`和`arr[mid]`，如果`arr[mid]`$>$`num`，则进入前半部分寻找，否则进入后半部分。这样搜索每个数的复杂度都是$O(\log(n))$
      
      
### 2）Important functions

这里只介绍两个比较重要的函数：`bisect.bisect_left()`和`bisect.bisect_right()/bisect.bisect()`，两者的唯一区别是，遇到相同的数值时，`bisect.bisect_left`会返回最左端的index，`bisect.bisect_right()/bisect.bisect()`会返回最右端的index+1

In [1]:
import bisect

a = [0,1,2,2,2,3,4]
bisect.bisect_left(a, 2), bisect.bisect(a, 2), bisect.bisect_right(a, 2)

(2, 5, 5)

In [2]:
bisect.bisect_left(a, 2.5)

5

In [3]:
bisect.bisect(a, 4.5)

7

### 3) bisect with list

如果需要在一个increasing array中插入一个元素并使得array仍然保持升序，可以将刚才的函数和`list`的`insert`方法结合使用

In [4]:
def list_insert(arr, num):
    ind = bisect.bisect_left(arr, num)
    arr.insert(ind, num)

a = [0,1,2,2,2,3,4]
list_insert(a, 2.1)
a

[0, 1, 2, 2, 2, 2.1, 3, 4]

In [5]:
list_insert(a, -1)
a

[-1, 0, 1, 2, 2, 2, 2.1, 3, 4]

In [6]:
list_insert(a, 5)
a

[-1, 0, 1, 2, 2, 2, 2.1, 3, 4, 5]

## 2 Sum Tree

### 1) Concept

Sum Tree有多个中文名称，我习惯叫它“线段树”。

Sum Tree的结构是二叉树，每个节点要么是叶子节点，要么有两个子节点。Sum Tree的叶子节点储存数，所有的非叶子节点的值是其对应的叶子节点的和，如下图所示：([reference : image link](https://www.fcodelabs.com/2019/03/18/Sum-Tree-Introduction/))

<img src="./imgs/SumTree1.png"  width="700" height="700" align="bottom" />

为了便于理解，可以认为叶子节点表示的是一个一个相连的区间，每个叶子节点的数值表示该区间的长度，此时则可以轻易地从根节点出发寻找任意数值所在的区间对应的叶子节点：([reference : image link](https://www.fcodelabs.com/2019/03/18/Sum-Tree-Introduction/))

<img src="./imgs/SumTree2.png"  width="700" height="700" align="bottom" />


### 2) Analysis

实现Sum Tree主要需要实现两个功能：查找和更新

查找即寻找一个数所在的位置，这个比较简单，直接比较左右的值然后进入相应的节点即可

更新则需要从叶子节点向上更新，由于每个节点是期两个子节点的和，只需计算变化量然后一点点向上更新至根节点即可


如果专门为了树的节点建一个类`TreeNode`，则该类需要四个属性：
  * left : 该节点的左子树
  * right : 该节点的右子树
  * val : 该节点对应的值
  * parent : 该节点的父节点

这是可行的，但是有更简单的方法，请看下面的代码：

In [7]:
class SumTree:
    
    def __init__(self, capacity):
        
        self.capacity = capacity
        # the first capacity-1 positions are not leaves
        self.vals = [0 for _ in range(2*capacity - 1)] # think about why if you are not familiar with this
        
    def retrive(self, num):
        '''
        This function find the first index whose cumsum is no smaller than num
        '''
        ind = 0 # search from root
        while ind < self.capacity-1: # not a leaf
            left = 2*ind + 1
            right = left + 1
            if num > self.vals[left]: # the sum of the whole left tree is not large enouth
                num -= self.vals[left] # think about why?
                ind = right
            else: # search in the left tree
                ind = left
        return ind - self.capacity + 1
    
    def update(self, delta, ind):
        '''
        Change the value at ind by delta, and update the tree
        Notice that this ind should be the index in real memory part, instead of the ind in self.vals
        '''
        ind += self.capacity - 1
        while True:
            self.vals[ind] += delta
            if ind == 0:
                break
            ind -= 1
            ind //= 2

### 计算Sum Tree需要的节点个数

假设一共有$n$个数据点，现在要计算Sum Tree一共有多少节点

先思考：完全二叉树的情况$n$满足的条件:
  * 如果数据在第$k$层，那么一共有$n=2^k$数据，Sum Tree一共有$2^{k+1}-1$节点
  
如果不是完全二叉树：
  * 存在$k$, s.t. $2^{k-1}<n, 2^k \ge n$，此时数据保存在第$k$层，也有可能在$k-1$层
  * 假设第$k-1$层有$x$个数据（意味着第$k$层有$n-x$个数据）
  * 前$k-1$层排满，有$2^k - 1$个节点，然后第$k$层有$n-x$个数据，一共是$2^k+n-x-1$个节点
  * $n-x+2x = 2^k, x = 2^k-n$，一共$2n-1$个节点

## 3 Duel Structure

Duel Structure本身并没有什么特别的，只是要注意文章中减去max的操作，代码如下

In [8]:
import torch
import torch.nn as nn
import torch.functional as F

class Q_Network(nn.Module):

    def __init__(self, state_size, action_size, hidden=[64, 64]):
        super(Q_Network, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden[0])
        self.fc2 = nn.Linear(hidden[0], hidden[1])
        self.fc3 = nn.Linear(hidden[1], action_size)
        self.fc4 = nn.Linear(hidden[1], 1)

    def forward(self, state):
        x = state
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x1 = self.fc3(x)
        x1 = x1 - torch.max(x1, dim=1, keepdim=True)[0] # set the max to be 0
        x2 = self.fc4(x)
        return x1 + x2